# Land Surface Temperature (LST) Mapping with Landsat 8/9

## Overview
This notebook demonstrates **Land Surface Temperature (LST) mapping** using satellite thermal infrared data from Landsat 8 and 9. LST is a critical parameter for understanding urban heat islands, vegetation health, and climate patterns.

### What You'll Learn:
1. **Theory**: LST retrieval methods and thermal remote sensing principles
2. **Data Acquisition**: Fetching Landsat thermal and optical data from Microsoft Planetary Computer
3. **Processing**: Converting Digital Numbers (DN) to brightness temperature and LST
4. **Analysis**: Correlating LST with vegetation indices (NDVI) and land cover
5. **Visualization**: Creating informative maps and statistical plots

### Key Terminology:
- **LST**: Land Surface Temperature - the radiative skin temperature of the ground
- **TIRS**: Thermal Infrared Sensor (Bands 10 & 11 on Landsat 8/9)
- **NDVI**: Normalized Difference Vegetation Index - indicator of vegetation health
- **UHI**: Urban Heat Island - phenomenon where urban areas are significantly warmer than surroundings

---

## 1. Setup and Imports

We'll use the following libraries:
- `pystac-client`: To search satellite data catalogs (STAC API)
- `odc-stac`: To load and process STAC items as xarray datasets
- `planetary-computer`: Microsoft's STAC catalog authentication
- `xarray` & `rioxarray`: For multi-dimensional geospatial array processing
- `matplotlib`: For visualization
- `geopandas` & `geopy`: For location handling

In [ ]:
# Data access and geospatial processing
import pystac_client
import planetary_computer
import odc.stac
import xarray as xr
import rioxarray  # For CRS and geospatial metadata
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim

# Visualization
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

# Utilities
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for better-looking plots
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ All libraries imported successfully!")

: 

---
## 2. Theoretical Background: LST Retrieval Methods

### 2.1 What is Land Surface Temperature?
LST is different from air temperature - it measures the **radiative temperature of the Earth's surface**, including soil, vegetation, water, and built structures. It's crucial for:
- Urban planning (identifying heat islands)
- Agriculture (crop stress monitoring)
- Climate studies (energy balance)
- Hydrology (evapotranspiration estimates)

### 2.2 Common LST Retrieval Algorithms

| Method | Description | Data Requirements | Complexity |
|--------|-------------|-------------------|------------|
| **Single-Channel Algorithm** | Uses one thermal band + atmospheric correction | 1 thermal band, emissivity | Low |
| **Split-Window Algorithm** | Uses differential absorption between 2 thermal bands | 2 thermal bands (e.g., B10 & B11) | Medium |
| **Mono-Window Algorithm** | Requires atmospheric parameters (transmissivity, etc.) | 1 thermal band, atmospheric data | Medium-High |
| **Radiative Transfer Equation (RTE)** | Physics-based, atmospheric correction | Atmospheric profiles, emissivity | High |

### 2.3 Landsat Collection 2 Approach (Used in This Notebook)

Landsat Collection 2 Level-2 products provide **pre-processed Surface Temperature** (ST) from Band 10 (TIRS-1). The data comes with:
- Atmospheric correction already applied
- Scale and offset factors for conversion

**Conversion Formula:**
```
LST (Kelvin) = DN × 0.00341802 + 149.0
LST (Celsius) = LST (Kelvin) - 273.15
```

Where:
- `DN` = Digital Number from Band 10 (ST_B10)
- `0.00341802` = Scale factor
- `149.0` = Offset

---

## 3. Define Study Area

We'll use **geocoding** to convert a city name to coordinates and create a bounding box for our analysis.

In [ ]:
def get_city_bbox(city_name, buffer_km=15):
    """
    Geocode a city and create a bounding box around it.
    
    Parameters:
    -----------
    city_name : str
        Name of the city (e.g., "Bangalore", "Phoenix, Arizona")
    buffer_km : float
        Buffer distance in kilometers around city center (default: 15km)
        
    Returns:
    --------
    tuple : (minx, miny, maxx, maxy) bounding box in EPSG:4326
    """
    # Initialize geocoder
    geolocator = Nominatim(user_agent="uhi_monitor")
    
    # Get location
    location = geolocator.geocode(city_name)
    if not location:
        raise ValueError(f"Could not find location: {city_name}")
    
    lat, lon = location.latitude, location.longitude
    print(f"📍 Found {city_name} at ({lat:.4f}, {lon:.4f})")
    
    # Convert km to degrees (approximate at equator: 1° ≈ 111 km)
    # Adjust for latitude using cosine correction
    lat_buffer = buffer_km / 111.0
    lon_buffer = buffer_km / (111.0 * np.cos(np.radians(lat)))
    
    bbox = (
        lon - lon_buffer,  # minx
        lat - lat_buffer,  # miny
        lon + lon_buffer,  # maxx
        lat + lat_buffer   # maxy
    )
    
    print(f"📦 Bounding box: {bbox}")
    return bbox


# CONFIGURE YOUR STUDY AREA HERE
CITY_NAME = "Phoenix, Arizona"  # Change this to your city of interest
DATE_START = "2024-06-01"       # Summer months typically show stronger UHI effects
DATE_END = "2024-08-31"
MAX_CLOUD_COVER = 15            # Maximum cloud cover percentage

# Get bounding box
bbox = get_city_bbox(CITY_NAME, buffer_km=20)

: 

---
## 4. Data Acquisition from Microsoft Planetary Computer

We'll search for Landsat Collection 2 Level-2 scenes that meet our criteria:
- **Collection**: `landsat-c2-l2` (atmospherically corrected)
- **Spatial**: Intersects our bounding box
- **Temporal**: Within our date range
- **Quality**: Low cloud cover

In [ ]:
def search_landsat_scenes(bbox, date_range, max_cloud_cover=20):
    """
    Search for Landsat Collection 2 Level-2 scenes.
    
    Parameters:
    -----------
    bbox : tuple
        Bounding box (minx, miny, maxx, maxy)
    date_range : tuple
        (start_date, end_date) as strings "YYYY-MM-DD"
    max_cloud_cover : float
        Maximum acceptable cloud cover percentage
        
    Returns:
    --------
    pystac.ItemCollection : Found scenes sorted by cloud cover
    """
    # Connect to Microsoft Planetary Computer STAC API
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,  # Handles authentication
    )
    
    print(f"🔍 Searching Landsat Collection 2 scenes...")
    print(f"   Date range: {date_range[0]} to {date_range[1]}")
    print(f"   Max cloud cover: {max_cloud_cover}%")
    
    # Search the catalog
    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox,
        datetime=f"{date_range[0]}/{date_range[1]}",
        query={"eo:cloud_cover": {"lt": max_cloud_cover}},
    )
    
    items = search.item_collection()
    print(f"✅ Found {len(items)} scenes")
    
    if len(items) == 0:
        raise ValueError("No scenes found. Try expanding date range or increasing cloud cover threshold.")
    
    # Sort by cloud cover (ascending)
    items_sorted = sorted(items, key=lambda x: x.properties.get("eo:cloud_cover", 100))
    
    # Display top 5 clearest scenes
    print("\n📊 Top 5 clearest scenes:")
    for i, item in enumerate(items_sorted[:5]):
        cloud_cover = item.properties.get("eo:cloud_cover", "N/A")
        date = item.properties.get("datetime", "N/A")[:10]
        platform = item.properties.get("platform", "N/A")
        print(f"   {i+1}. {date} | {platform} | Cloud cover: {cloud_cover:.1f}%")
    
    return items_sorted


# Execute search
scenes = search_landsat_scenes(
    bbox=bbox,
    date_range=(DATE_START, DATE_END),
    max_cloud_cover=MAX_CLOUD_COVER
)

# Select the clearest scene
best_scene = scenes[0]
print(f"\n🎯 Selected scene: {best_scene.id}")
print(f"   Date: {best_scene.properties['datetime'][:10]}")
print(f"   Cloud cover: {best_scene.properties['eo:cloud_cover']:.1f}%")

---
## 5. Load and Process Satellite Data

We'll load the following bands:
- **ST_B10**: Surface Temperature from thermal band 10
- **SR_B4**: Red band (for NDVI calculation)
- **SR_B5**: Near-Infrared band (for NDVI calculation)
- **QA_PIXEL**: Quality assessment band (for cloud masking)

In [ ]:
def load_landsat_data(scene, bbox, resolution=100):
    """
    Load Landsat bands as an xarray Dataset.
    
    Parameters:
    -----------
    scene : pystac.Item
        STAC item representing the satellite scene
    bbox : tuple
        Bounding box to clip the data
    resolution : int
        Spatial resolution in meters (default: 100m for faster processing)
        
    Returns:
    --------
    xarray.Dataset : Multi-band satellite data
    """
    print(f"📥 Loading satellite data at {resolution}m resolution...")
    
    # Load data using odc-stac
    data = odc.stac.load(
        [scene],
        bands=["ST_B10", "SR_B4", "SR_B5", "QA_PIXEL"],  # Thermal, Red, NIR, QA
        bbox=bbox,
        resolution=resolution,
        chunks={"x": 2048, "y": 2048},  # Enable dask for large datasets
    )
    
    print(f"✅ Data loaded: {data.dims['x']} x {data.dims['y']} pixels")
    print(f"   Bands: {list(data.data_vars)}")
    
    return data


# Load the data
data = load_landsat_data(best_scene, bbox, resolution=100)

# Display dataset info
print("\n📋 Dataset Information:")
print(data)

---
## 6. Calculate Land Surface Temperature (LST)

### Processing Steps:
1. **Extract thermal band** (ST_B10)
2. **Apply scale and offset** to convert DN to Kelvin
3. **Convert to Celsius** for interpretability
4. **Mask invalid values** (e.g., clouds, water, fill values)

In [ ]:
def calculate_lst(data):
    """
    Calculate Land Surface Temperature from Landsat ST_B10.
    
    Landsat Collection 2 Surface Temperature formula:
    LST (K) = DN × 0.00341802 + 149.0
    
    Parameters:
    -----------
    data : xarray.Dataset
        Dataset containing ST_B10 band
        
    Returns:
    --------
    xarray.DataArray : LST in degrees Celsius
    """
    print("Calculating Land Surface Temperature...")
    
    # Extract thermal band
    thermal_band = data["ST_B10"]
    
    # Apply Landsat Collection 2 scaling factors
    SCALE_FACTOR = 0.00341802
    OFFSET = 149.0
    
    # Calculate LST in Kelvin
    lst_kelvin = (thermal_band * SCALE_FACTOR) + OFFSET
    
    # Convert to Celsius
    lst_celsius = lst_kelvin - 273.15
    
    # Mask unrealistic values (valid range: -50°C to 70°C)
    lst_celsius = lst_celsius.where(
        (lst_celsius > -50) & (lst_celsius < 70),
        np.nan
    )
    
    # Add metadata
    lst_celsius.attrs['long_name'] = 'Land Surface Temperature'
    lst_celsius.attrs['units'] = 'degrees Celsius'
    
    print(f"   Min LST: {float(lst_celsius.min()):.2f}°C")
    print(f"   Max LST: {float(lst_celsius.max()):.2f}°C")
    print(f"   Mean LST: {float(lst_celsius.mean()):.2f}°C")
    
    return lst_celsius


# Calculate LST
lst = calculate_lst(data)

---
## 7. Calculate Vegetation Index (NDVI)

**NDVI (Normalized Difference Vegetation Index)** is calculated as:

$$
NDVI = \frac{NIR - Red}{NIR + Red}
$$

### Interpretation:
- **-1 to 0**: Water, clouds, snow
- **0 to 0.2**: Bare soil, sand, rock
- **0.2 to 0.4**: Sparse vegetation (grassland, shrubs)
- **0.4 to 0.7**: Moderate to dense vegetation
- **0.7 to 1.0**: Very dense, healthy vegetation (forests)

NDVI helps us understand the cooling effect of vegetation on LST.

In [ ]:
def calculate_ndvi(data):
    """
    Calculate Normalized Difference Vegetation Index (NDVI).
    
    NDVI = (NIR - Red) / (NIR + Red)
    
    Parameters:
    -----------
    data : xarray.Dataset
        Dataset containing SR_B4 (Red) and SR_B5 (NIR)
        
    Returns:
    --------
    xarray.DataArray : NDVI values (-1 to 1)
    """
    print("🌿 Calculating NDVI...")
    
    # Extract and scale bands (Landsat C2 scale factor: 0.0000275, offset: -0.2)
    red = data["SR_B4"] * 0.0000275 - 0.2
    nir = data["SR_B5"] * 0.0000275 - 0.2
    
    # Calculate NDVI
    ndvi = (nir - red) / (nir + red)
    
    # Mask invalid values
    ndvi = ndvi.where((ndvi >= -1) & (ndvi <= 1), np.nan)
    
    # Add metadata
    ndvi.attrs['long_name'] = 'Normalized Difference Vegetation Index'
    ndvi.attrs['units'] = 'dimensionless'
    
    print(f"   Min NDVI: {float(ndvi.min()):.3f}")
    print(f"   Max NDVI: {float(ndvi.max()):.3f}")
    print(f"   Mean NDVI: {float(ndvi.mean()):.3f}")
    
    return ndvi


# Calculate NDVI
ndvi = calculate_ndvi(data)

---
## 8. Apply Cloud Masking (Quality Control)

The `QA_PIXEL` band contains bit-packed quality flags. We'll mask out:
- Clouds (high confidence)
- Cloud shadows
- Water bodies (optional - comment out if you want to see water temperature)

In [ ]:
def apply_cloud_mask(lst, ndvi, qa_band):
    """
    Apply quality mask to remove clouds and other artifacts.
    
    QA_PIXEL bit flags (Landsat Collection 2):
    - Bit 3: Cloud
    - Bit 4: Cloud shadow
    - Bit 7: Water
    
    Parameters:
    -----------
    lst : xarray.DataArray
        Land Surface Temperature
    ndvi : xarray.DataArray
        Vegetation index
    qa_band : xarray.DataArray
        Quality assessment band
        
    Returns:
    --------
    tuple : (masked_lst, masked_ndvi)
    """
    print("🎭 Applying cloud and water masks...")
    
    # Extract bit flags
    cloud = (qa_band & (1 << 3)) != 0       # Bit 3: Cloud
    shadow = (qa_band & (1 << 4)) != 0      # Bit 4: Cloud shadow
    water = (qa_band & (1 << 7)) != 0       # Bit 7: Water
    
    # Create combined mask (True = keep, False = mask out)
    valid_mask = ~(cloud | shadow | water)
    
    # Apply mask
    lst_masked = lst.where(valid_mask)
    ndvi_masked = ndvi.where(valid_mask)
    
    # Calculate percentage of valid pixels
    valid_percentage = (valid_mask.sum() / valid_mask.size * 100).values
    print(f"   Valid pixels: {valid_percentage:.1f}%")
    
    return lst_masked, ndvi_masked


# Apply masking
lst_clean, ndvi_clean = apply_cloud_mask(lst, ndvi, data["QA_PIXEL"])

---
## 9. Visualization: LST and NDVI Maps

Create side-by-side visualizations to compare temperature and vegetation patterns.

In [ ]:
def plot_lst_ndvi_comparison(lst, ndvi, city_name):
    """
    Create a professional side-by-side comparison of LST and NDVI.
    
    Parameters:
    -----------
    lst : xarray.DataArray
        Land Surface Temperature (°C)
    ndvi : xarray.DataArray
        Vegetation index
    city_name : str
        Name of the study area
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # === LEFT PANEL: Land Surface Temperature ===
    im1 = lst.plot(
        ax=ax1,
        cmap='RdYlBu_r',  # Red (hot) to Blue (cool)
        vmin=lst.quantile(0.02),  # Clip extreme outliers for better visualization
        vmax=lst.quantile(0.98),
        cbar_kwargs={
            'label': 'Temperature (°C)',
            'shrink': 0.8,
            'pad': 0.02
        },
        add_labels=False
    )
    ax1.set_title(f'Land Surface Temperature\n{city_name}', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.grid(True, alpha=0.3, linestyle='--')
    
    # === RIGHT PANEL: NDVI ===
    im2 = ndvi.plot(
        ax=ax2,
        cmap='RdYlGn',  # Red (bare) to Green (vegetation)
        vmin=-0.2,
        vmax=0.8,
        cbar_kwargs={
            'label': 'NDVI (Vegetation Health)',
            'shrink': 0.8,
            'pad': 0.02
        },
        add_labels=False
    )
    ax2.set_title(f'Vegetation Index (NDVI)\n{city_name}', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('')
    ax2.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n📊 Summary Statistics:")
    print("\nLand Surface Temperature:")
    print(f"   Mean: {float(lst.mean()):.2f}°C")
    print(f"   Std Dev: {float(lst.std()):.2f}°C")
    print(f"   Range: {float(lst.min()):.2f}°C to {float(lst.max()):.2f}°C")
    
    print("\nVegetation Index:")
    print(f"   Mean NDVI: {float(ndvi.mean()):.3f}")
    print(f"   Vegetated area (NDVI > 0.3): {float((ndvi > 0.3).sum() / ndvi.count() * 100):.1f}%")


# Create visualization
plot_lst_ndvi_comparison(lst_clean, ndvi_clean, CITY_NAME)

---
## 10. Statistical Analysis: LST vs. NDVI Correlation

Analyze the inverse relationship between vegetation and temperature:
- **Hypothesis**: Areas with high vegetation (high NDVI) should have lower LST
- **Urban Heat Island Effect**: Urban areas (low NDVI) tend to be hotter

In [ ]:
def analyze_lst_ndvi_relationship(lst, ndvi):
    """
    Analyze the statistical relationship between LST and NDVI.
    
    Creates:
    1. Scatter plot with density coloring
    2. Binned bar chart showing mean LST by vegetation category
    3. Correlation coefficient
    """
    # Flatten arrays and remove NaN values
    lst_flat = lst.values.flatten()
    ndvi_flat = ndvi.values.flatten()
    
    # Create mask for valid data
    valid = ~(np.isnan(lst_flat) | np.isnan(ndvi_flat))
    lst_valid = lst_flat[valid]
    ndvi_valid = ndvi_flat[valid]
    
    # Calculate correlation
    correlation = np.corrcoef(lst_valid, ndvi_valid)[0, 1]
    
    # === VISUALIZATION ===
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # LEFT: Scatter plot with hexbin for density
    hb = ax1.hexbin(
        ndvi_valid,
        lst_valid,
        gridsize=50,
        cmap='YlOrRd',
        mincnt=1,
        edgecolors='none'
    )
    ax1.set_xlabel('NDVI (Vegetation Index)', fontsize=12)
    ax1.set_ylabel('LST (°C)', fontsize=12)
    ax1.set_title(f'LST vs. NDVI Relationship\nCorrelation: {correlation:.3f}', 
                  fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3, linestyle='--')
    
    # Add colorbar
    cb = plt.colorbar(hb, ax=ax1, label='Pixel Density')
    
    # Add trend line
    z = np.polyfit(ndvi_valid, lst_valid, 1)
    p = np.poly1d(z)
    ndvi_range = np.linspace(ndvi_valid.min(), ndvi_valid.max(), 100)
    ax1.plot(ndvi_range, p(ndvi_range), "r--", linewidth=2, 
             label=f'Trend: LST = {z[0]:.2f}×NDVI + {z[1]:.2f}')
    ax1.legend(loc='upper right')
    
    # RIGHT: Binned analysis
    # Create NDVI bins
    bins = [-1, 0, 0.2, 0.4, 0.6, 1.0]
    labels = ['Water/Cloud', 'Bare Soil', 'Sparse Veg', 'Moderate Veg', 'Dense Veg']
    
    # Digitize NDVI values into bins
    ndvi_binned = np.digitize(ndvi_valid, bins) - 1
    
    # Calculate mean LST for each bin
    mean_lst_by_bin = []
    std_lst_by_bin = []
    for i in range(len(labels)):
        mask = ndvi_binned == i
        if mask.sum() > 0:
            mean_lst_by_bin.append(lst_valid[mask].mean())
            std_lst_by_bin.append(lst_valid[mask].std())
        else:
            mean_lst_by_bin.append(0)
            std_lst_by_bin.append(0)
    
    # Bar plot
    colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60']
    bars = ax2.bar(labels, mean_lst_by_bin, color=colors, alpha=0.7, 
                   yerr=std_lst_by_bin, capsize=5, edgecolor='black', linewidth=1.5)
    ax2.set_xlabel('Land Cover Type (NDVI Category)', fontsize=12)
    ax2.set_ylabel('Mean LST (°C)', fontsize=12)
    ax2.set_title('Temperature by Vegetation Category', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='x', rotation=15)
    ax2.grid(True, axis='y', alpha=0.3, linestyle='--')
    
    # Add value labels on bars
    for bar, value in zip(bars, mean_lst_by_bin):
        if value > 0:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{value:.1f}°C',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("\nCorrelation Analysis:")
    print(f"   Pearson correlation: {correlation:.3f}")
    print(f"   Interpretation: {'Strong' if abs(correlation) > 0.7 else 'Moderate' if abs(correlation) > 0.4 else 'Weak'} "
          f"{'negative' if correlation < 0 else 'positive'} relationship")
    
    print("\nTemperature Difference (Urban Heat Island Effect):")
    urban_temp = mean_lst_by_bin[1]  # Bare soil (urban)
    veg_temp = mean_lst_by_bin[-1]   # Dense vegetation
    uhi_magnitude = urban_temp - veg_temp
    print(f"   Urban (bare soil) mean LST: {urban_temp:.2f}°C")
    print(f"   Vegetated area mean LST: {veg_temp:.2f}°C")
    print(f"   UHI magnitude: {uhi_magnitude:.2f}°C")


# Run analysis
analyze_lst_ndvi_relationship(lst_clean, ndvi_clean)

---
## 11. Export Results (Optional)

Save processed LST and NDVI as GeoTIFF files for use in GIS software.

In [ ]:
def export_geotiff(data_array, output_path, description):
    """
    Export xarray DataArray to GeoTIFF.
    
    Parameters:
    -----------
    data_array : xarray.DataArray
        Data to export
    output_path : str
        Output file path (e.g., 'lst_output.tif')
    description : str
        Description of the data
    """
    print(f"💾 Exporting {description} to {output_path}...")
    
    # Ensure CRS is set
    if data_array.rio.crs is None:
        data_array = data_array.rio.write_crs("EPSG:32633")  # UTM zone (adjust as needed)
    
    # Write to file
    data_array.rio.to_raster(output_path, driver="GTiff", compress="LZW")
    print(f"✅ Export complete!")


# Uncomment to export:
# export_geotiff(lst_clean, f"LST_{CITY_NAME.replace(' ', '_')}.tif", "Land Surface Temperature")
# export_geotiff(ndvi_clean, f"NDVI_{CITY_NAME.replace(' ', '_')}.tif", "NDVI")

print("ℹ️ Export section ready (uncomment lines above to save GeoTIFFs)")

---
## 12. Summary and Key Findings

### What We Accomplished:
1. ✅ **Acquired** cloud-free Landsat 8/9 thermal and optical imagery
2. ✅ **Calculated** Land Surface Temperature using Collection 2 Surface Temperature product
3. ✅ **Computed** NDVI to assess vegetation health
4. ✅ **Analyzed** the inverse relationship between vegetation and temperature
5. ✅ **Quantified** the Urban Heat Island effect

### Typical Findings:
- **Urban areas** (low NDVI) are typically 5-10°C warmer than vegetated areas
- **Parks and green spaces** act as "cool islands" within cities
- **Water bodies** often have moderate temperatures (if not masked)
- **Bare soil/concrete** shows the highest surface temperatures

### Applications:
- 🏙️ **Urban planning**: Identify heat hotspots for targeted greening
- 🌳 **Environmental monitoring**: Track vegetation stress and degradation
- 🏥 **Public health**: Assess heat exposure risks for vulnerable populations
- 🌾 **Agriculture**: Monitor crop health and irrigation efficiency

---

## Next Steps:
- **Time series analysis**: Compare LST across multiple seasons or years
- **Advanced methods**: Implement Split-Window or Mono-Window algorithms
- **Machine learning**: Predict LST from other environmental variables
- **Integration**: Combine with weather station data for validation

---

## References:
1. USGS Landsat Collection 2 Surface Temperature: https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-temperature
2. Li et al. (2013). "Satellite-derived land surface temperature: Current status and perspectives." *Remote Sensing of Environment*.
3. Sobrino et al. (2004). "Land surface temperature retrieval from LANDSAT TM 5." *Remote Sensing of Environment*.
4. Microsoft Planetary Computer STAC API: https://planetarycomputer.microsoft.com/

---

**Author**: UHI-Monitor Project  
**Last Updated**: January 2026  
**License**: MIT